## Import packages

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt 
%matplotlib inline

## Import Data and DataLoader

In [3]:


transforms_func = transforms.Compose([
    transforms.RandomResizedCrop(416), 
    transforms.RandomHorizontalFlip(), 
    transforms.RandomVerticalFlip(),   
    transforms.RandomRotation(15),     
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
    transforms.Resize((416, 416)),      
    transforms.ToTensor(),             
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the training dataset with augmentations
train_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets/train',
    transform=transforms_func
)

# Define the validation transformation (typically only resizing and normalization)
val_transforms = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets/val',
    transform=val_transforms
)



data_dir = 'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets'
batch_size = 32
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)


## Model Building

In [5]:

# Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Define MobileNet
class MobileNet(nn.Module):
    def __init__(self, num_classes=25):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 128, stride=1),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 256, stride=1),
            DepthwiseSeparableConv(256, 512, stride=2),

            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),

            DepthwiseSeparableConv(512, 1024, stride=2),
            DepthwiseSeparableConv(1024, 1024, stride=1),

            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


## Training Pipeline / Loop

In [6]:

# Training function with gradient accumulation
def train_mobilenet(num_classes, epochs, learning_rate, train_dataloader, val_dataloader, model_path, accumulation_steps=4):
    # Initialize model, loss function, and optimizer
    model = MobileNet(num_classes=num_classes).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loader_tqdm = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}')
        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loader_tqdm.set_postfix(loss=running_loss/((i+1)/accumulation_steps), accuracy=100*correct/total)
        
        train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct / total
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}/{epochs}'):
                inputs, labels = inputs.cuda(), labels.cuda()
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_dataloader)
        val_accuracy = 100 * val_correct / val_total
        
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    
    # Save the trained model
    torch.save(model.state_dict(), model_path)

## Hyper parameter and Training

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:

train_mobilenet(
    num_classes=25,
    epochs=10,
    learning_rate=0.001,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model_path='mobilenet_model.pth',
    accumulation_steps=4  # Accumulate gradients over 4 steps
)

retraining


In [ ]:


def train_mobilenet(num_classes, epochs, learning_rate, train_dataloader, val_dataloader, model_path, accumulation_steps=4):
    # Initialize model, loss function, and optimizer
    model = MobileNet(num_classes=25).cuda()
    model.load_state_dict(torch.load('mobilenet_model(1).pth'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loader_tqdm = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}')
        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.cuda(non_blocking=True), labels.cuda(non_blocking=True)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loader_tqdm.set_postfix(loss=running_loss/((i+1)/accumulation_steps), accuracy=100*correct/total)
        
        train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct / total
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}/{epochs}'):
                inputs, labels = inputs.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_dataloader)
        val_accuracy = 100 * val_correct / val_total
        
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    
    # Save the trained model
    torch.save(model.state_dict(), model_path)

# Example usage
if __name__ == "__main__":
    # Data transforms
    transforms_func = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),   
        
        # transforms.RandomRotation(15),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transforms = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_datasets = datasets.ImageFolder(
        'C:\\Users\\sshak\\Downloads\\Seen Datasets2\\Seen Datasets/train',
        transform=transforms_func
    )
    val_datasets = datasets.ImageFolder(
        'C:\\Users\\sshak\\Downloads\\Seen Datasets2\\Seen Datasets/val',
        transform=val_transforms
    )

    batch_size = 32
    train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)



In [ ]:

train_mobilenet(
        num_classes=25,
        epochs=10,
        learning_rate=0.001,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        model_path='mobilenet_model(2).pth',
        accumulation_steps=4  # Accumulate gradients over 4 steps
    )